In [1]:
import requests
import time
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 로드합니다
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_script(url):
    api_endpoint = "http://127.0.0.1:8010/extract_info"
    response = requests.get(api_endpoint, params={"url": url})
    return response.json()

In [3]:
start_time = time.time()
result = get_script("https://youtu.be/ZueGqZ6Elfo?si=rvvch1GUNnSyg3PT")
print(f"실행시간: {time.time() - start_time} seconds")

실행시간: 11.530746459960938 seconds


In [4]:
import json
import os
if os.path.exists('script.json'):
    os.remove('script.json')

with open('script.json', 'w', encoding='utf-8') as json_file:
    json.dump(result['script'], json_file, ensure_ascii=False, indent=4)

In [4]:
import json
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter , RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain import OpenAI, LLMChain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback
from langchain import hub

prompt = hub.pull("teddynote/summary-stuff-documents-korean")
prompt.pretty_print()


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0,
    callbacks=[StreamingCallback()],
)

# Step 1: 파일 로드 및 데이터 변환
def load_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        script_data = json.load(file)
    return script_data

/home/jinu/miniconda3/envs/youtubee/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"



'Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n'

In [24]:
import json
from langchain_community.document_loaders import JSONLoader

# 파일을 UTF-8로 읽어들이기
file_path = 'script.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 임시 파일로 저장 (선택 사항: 만약 기존 파일을 유지하고 싶을 경우)
temp_file_path = 'temp_script.json'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    json.dump(data, temp_file, ensure_ascii=False, indent=4)

# JSONLoader로 임시 파일 로드
jq_schema = '.[] | {content: .text, start_time: .start, end_time: .end}'
docs = JSONLoader(file_path=temp_file_path, jq_schema=jq_schema,text_content=False).load()

# 각 Document의 page_content를 디코딩하여 한글로 변환
for doc in docs:
    # page_content를 JSON 객체로 변환 후 문자열 디코딩
    decoded_content = json.loads(doc.page_content) 
    doc.page_content = decoded_content # JSON 문자열 -> Python dict 변환



In [25]:
docs[0]

Document(metadata={'source': '/home/jinu/my_ws/youtube_script_chatbot/temp_script.json', 'seq_num': 1}, page_content={'content': '전 감자이구요 직장인을 위한 인공지능 오늘은 비슷한 이미지를 만드는 방법 알려드리려고 합니다', 'start_time': 0, 'end_time': 8.22})

In [26]:
stuff_chain = create_stuff_documents_chain(llm, prompt)
answer = stuff_chain.invoke({"context": docs})

- 🥔 전 감자이며 직장인을 위한 인공지능을 소개합니다.
- 🎨 비슷한 이미지를 만드는 방법을 알려드립니다.
- 💻 사용할 서비스는 채트 GPT와 코파일럿입니다.
- 🔍 구글의 이메진3 서비스도 소개합니다.
- 📥 이미지를 업로드하고 비슷한 이미지를 요청하는 방법을 설명합니다.
- 🖼️ 채트 GPT와 코파일럿 모두 비슷한 이미지를 생성할 수 있습니다.
- 🔄 프롬프트를 사용하여 더 구체적인 이미지를 만들 수 있습니다.
- 🆓 구글의 이메진3는 무료로 사용할 수 있습니다.
- ✏️ 프롬프트를 조정하여 원하는 이미지를 생성할 수 있습니다.
- 🌟 미드전이와 같은 다른 서비스도 활용할 수 있습니다.
- 👍 오늘 배운 방법으로 비슷한 이미지를 만드는 데 도움이 되길 바랍니다.

In [6]:
import json
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from sentence_transformers import SentenceTransformer, util

# 1. Sentence-BERT 모델 불러오기 (유사도 계산을 위해)
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. JSON 파일을 로드하고 `Document` 객체로 변환
file_path = 'temp_script.json'  # 파일 경로는 필요에 따라 조정
jq_schema = '.[] | {content: .text, start_time: .start, end_time: .end}'
docs = JSONLoader(file_path=file_path, jq_schema=jq_schema, text_content=False).load()

# 3. JSON 데이터를 읽어와 각 문단을 Document 객체에 저장
for doc in docs:
    doc.page_content = json.loads(doc.page_content)

# 4. 요약본 예시
summary = [
    "🥔 전 감자이며 직장인을 위한 인공지능을 소개합니다.",
    "🎨 비슷한 이미지를 만드는 방법을 알려드립니다.",
    "💻 사용할 서비스는 채트 GPT와 코파일럿입니다.",
    "🔍 구글의 이메진3 서비스도 소개합니다.",
    "📥 이미지를 업로드하고 비슷한 이미지를 요청하는 방법을 설명합니다.",
    "🖼️ 채트 GPT와 코파일럿 모두 비슷한 이미지를 생성할 수 있습니다.",
    "🔄 프롬프트를 사용하여 더 구체적인 이미지를 만들 수 있습니다.",
    "🆓 구글의 이메진3는 무료로 사용할 수 있습니다.",
    "✏️ 프롬프트를 조정하여 원하는 이미지를 생성할 수 있습니다.",
    "🌟 미드전이와 같은 다른 서비스도 활용할 수 있습니다.",
    "👍 오늘 배운 방법으로 비슷한 이미지를 만드는 데 도움이 되길 바랍니다."
]

# 5. 원문과 요약 간의 유사도 계산
# 모든 원문 문장을 하나의 리스트로 결합
original_sentences = [doc.page_content["content"] for doc in docs]

# 원문과 요약을 각각 임베딩 벡터로 변환
original_embeddings = model.encode(original_sentences, convert_to_tensor=True)
summary_embeddings = model.encode(summary, convert_to_tensor=True)

# 요약과 원문 간의 유사도를 계산 (cosine similarity)
similarities = util.pytorch_cos_sim(summary_embeddings, original_embeddings)

# 6. 유사도를 바탕으로 요약과 원문 매핑 생성
summary_to_original = {}
for idx, summary_text in enumerate(summary):
    # summary_text와 가장 유사한 원문의 인덱스 추출
    most_similar_idx = similarities[idx].argmax().item()
    most_similar_original = original_sentences[most_similar_idx]
    summary_to_original[summary_text] = most_similar_original

# 7. 각 Document 객체에 해당하는 요약 추가
for doc in docs:
    doc_summary = []
    for sum_text, original_text in summary_to_original.items():
        if original_text in doc.page_content['content']:
            doc_summary.append(sum_text)
    # 각 Document 객체에 "summary" 필드 추가
    doc.metadata["summary"] = doc_summary

# 8. 요약과 원문 매핑 결과 출력
for doc in docs:
    print(f"Original: {doc.page_content['content']}\nSummary: {doc.metadata['summary']}\n")


/home/jinu/miniconda3/envs/youtubee/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Original: 전 감자이구요 직장인을 위한 인공지능 오늘은 비슷한 이미지를 만드는 방법 알려드리려고 합니다
Summary: ['🥔 전 감자이며 직장인을 위한 인공지능을 소개합니다.']

Original: 앞에 영상에서 우리는 인공지능 서비스들로 쉽게 이미지 만드는 방법을 알려드렸는데요
Summary: []

Original: 이번에는 내가 원하는 이미지와 비슷한 이미지를 만들고 이를 가공하는 방법을 알려드리겠습니다
Summary: ['✏️ 프롬프트를 조정하여 원하는 이미지를 생성할 수 있습니다.']

Original: 일단 오늘 우리가 사용할 서비스는 채트 GPT 있구요
Summary: []

Original: 코파일럿 있습니다
Summary: []

Original: 참고로 코파일럿이 뭔가 바뀌는 것 같아요
Summary: []

Original: 우리 이거 관심있게 한번 지켜보도록 하죠
Summary: []

Original: 그리고 구글에서 이메젠3 라고 하는 서비스가 있습니다
Summary: ['🔍 구글의 이메진3 서비스도 소개합니다.']

Original: 이런 서비스인데요
Summary: []

Original: 재미나이에서 바로 하지 말고
Summary: []

Original: Sign up to try on imagefx 여기에 들어가서
Summary: []

Original: 구글 아이디로 로그인을 합니다
Summary: []

Original: 자 이런 화면이 나왔습니다
Summary: []

Original: 우리는 이미지 fx에서 이메젠3를 쓸 거고요
Summary: []

Original: 그리고 제가 개인적으로 봤을 때 이미지 생성의 최고봉인 미드전이도 소개시켜 드리려고 합니다
Summary: []

Original: 자 그러면 채치피티, 코파일럿, 이메이젠3 그리고 미드전이까지 오늘 쓸 건데
Summary: []

Original: 제가 이런 이미지와 비슷한 이미지를 만들고 싶어요
Summary: []

Original: 그러면

In [70]:
import json
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter 

# Step 1: 파일 로드 및 데이터 변환
def load_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        script_data = json.load(file)
    return script_data

# Step 2: LangChain에서 사용할 Document 객체 생성
def create_documents(script_data):
    documents = []
    for entry in script_data:
        # 청킹을 위해 텍스트 분리
        text_splitter = CharacterTextSplitter(chunk_size=64, chunk_overlap=16)
        split_texts = text_splitter.split_text(entry["text"])
        for text in split_texts:
            documents.append(
                Document(
                    page_content=text, 
                    metadata={"start": entry["start"], "end": entry["end"]}
                )
            )
    print(f"Total Documents Created: {len(documents)}")  # 문서 개수 확인
    return documents

# Step 3: Embedding 및 Retriever 설정
def setup_retriever(documents, api_key):
    # OpenAI 임베딩을 사용하여 문서 벡터화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=api_key)
    vectorstore = FAISS.from_documents(documents, embeddings)
        
    # as_retriever 메서드를 사용하여 Retriever 생성
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
    return retriever

# Step 4: LangChain QA 체인 설정
def setup_qa_chain(retriever):
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

# Step 5: 사용자 질의에 대한 응답 생성
def get_response(qa_chain, query):
    response = qa_chain.invoke(query)
    return response

# Main 실행 코드
script_path = "script.json"  # 업로드한 파일의 경로

# Step 1: 스크립트 파일 로드
script_data = load_script(script_path)

# Step 2: LangChain Document 객체 생성
documents = create_documents(script_data)

# Step 3: Retriever 설정 (API Key 전달)
retriever = setup_retriever(documents, api_key)

# Step 4: QA 체인 설정 (API Key 전달)
qa_chain = setup_qa_chain(retriever)

Total Documents Created: 108


In [72]:
# 사용자 질의에 대한 응답 생성 전 관련 문서 확인
user_query = "주제가 뭔가요?"
relevant_docs = retriever.invoke(user_query)
print(f"Relevant Documents for '{user_query}': {[doc.page_content for doc in relevant_docs]}")  # 관련 문서 확인
# Step 5: 질의에 대한 응답 생성
response = get_response(qa_chain, user_query)
print(f"Chatbot Response: {response['result']}")


Relevant Documents for '주제가 뭔가요?': ['비슷하죠?', '어떻게 하느냐', '심지어 무료입니다', '비슷합니다.', '참고로 코파일럿이 뭔가 바뀌는 것 같아요', '스타라이트 함 해볼까요', '코파일럿 있습니다', '이런 서비스인데요', '우리 이거 관심있게 한번 지켜보도록 하죠', '그럼 어떻게 됩니까']
Chatbot Response: 주제에 대한 정보는 제공되지 않았습니다. 그래서 주제가 무엇인지 알 수 없습니다.


In [85]:
import json
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

# Step 1: 대주제 및 소주제 추출 함수
def extract_topics(text):
    # OpenAI 모델을 사용하여 대주제 추출
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    main_topic_prompt = f"Extract the main topic of the following text in korean:\n\n{text}"
    main_topic = llm.predict(main_topic_prompt)

    # 소주제 추출
    subtopic_prompt = f"Extract subtopics for each section of the following text in korean:\n\n{text}"
    subtopics = llm.predict(subtopic_prompt)

    return main_topic.strip(), subtopics.strip()


# Step 2: 파일 로드 및 데이터 변환
def load_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        script_data = json.load(file)
    return script_data


# Step 3: LangChain에서 사용할 Document 객체 생성 (대주제 및 소주제를 본문에 추가)
def create_documents(script_data):
    documents = []
    for entry in script_data:
        text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=32)
        split_texts = text_splitter.split_text(entry["text"])

        # 대주제 및 소주제 추출
        main_topic, subtopics = extract_topics(entry["text"])
        subtopics_list = subtopics.split('\n')

        for idx, text in enumerate(split_texts):
            # 각 청킹된 텍스트 앞에 대주제 및 소주제 추가
            updated_text = f"Main Topic: {main_topic}\nSubtopic: {subtopics_list[idx] if idx < len(subtopics_list) else 'N/A'}\n\n{text}"
            documents.append(
                Document(
                    page_content=updated_text, 
                    metadata={"start": entry["start"], "end": entry["end"]}
                )
            )
    print(f"Total Documents Created: {len(documents)}")  # 문서 개수 확인
    return documents

# Step 5: Embedding 및 Retriever 설정
def setup_retriever(documents):
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
    return retriever

# Step 6: LangChain QA 체인 설정
def setup_qa_chain(retriever):
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

# Step 7: 사용자 질의에 대한 응답 생성
def get_response(qa_chain, query):
    response = qa_chain.invoke(query)
    return response

In [86]:
# Main 실행 코드
script_path = "script.json"  # 업로드한 파일의 경로

# Step 1: 스크립트 파일 로드
script_data = load_script(script_path)

# Step 2: LangChain Document 객체 생성 (대주제 및 소주제 포함)
documents = create_documents(script_data)

# Step 3: Retriever 설정 (API Key 전달)
retriever = setup_retriever(documents)

# Step 4: QA 체인 설정 (API Key 전달)
qa_chain = setup_qa_chain(retriever)

Total Documents Created: 108


In [87]:
# 사용자 질의에 대한 응답 생성 전 관련 문서 확인
user_query = "주제가 뭔가요?"
relevant_docs = retriever.invoke(user_query)
print(f"Relevant Documents for '{user_query}': {[doc.page_content for doc in relevant_docs]}")  # 관련 문서 확인
# Step 5: 질의에 대한 응답 생성
response = get_response(qa_chain, user_query)
print(f"Chatbot Response: {response['result']}")


Relevant Documents for '주제가 뭔가요?': ['Main Topic: 주제: 관심사에 대한 주의 깊은 관찰\nSubtopic: 주제: 관심 있는 사항\n\n우리 이거 관심있게 한번 지켜보도록 하죠', 'Main Topic: 주제: 유사성\nSubtopic: 텍스트 "비슷하죠?"는 매우 짧고 단순한 질문으로, 특정한 섹션이나 주제를 제공하지 않기 때문에 subtopics를 추출하기 어렵습니다. 만약 더 긴 텍스트나 구체적인 주제를 제공해 주신다면, 그에 맞는 subtopics를 생성해 드릴 수 있습니다. 추가적인 내용을 제공해 주시면 감사하겠습니다!\n\n비슷하죠?', 'Main Topic: 주제: 관계 또는 상호작용\nSubtopic: 주어진 텍스트는 "얘한테 넣게 되고"로, 문맥이 부족하여 구체적인 주제나 섹션을 파악하기 어렵습니다. 추가적인 정보나 문장이 제공된다면, 그에 맞춰 주제를 추출해드리겠습니다. 더 많은 내용을 제공해주시겠어요?\n\n얘한테 넣게 되고', 'Main Topic: 주제: 질문 또는 상황에 대한 궁금증\nSubtopic: 제공하신 텍스트 "그럼 어떻게 됩니까"는 매우 짧아서 구체적인 섹션이나 주제를 포함하고 있지 않습니다. 좀 더 긴 텍스트나 특정 내용을 제공해 주시면, 그에 따라 섹션과 부주제를 추출해 드릴 수 있습니다. 추가 정보를 주시면 도와드리겠습니다!\n\n그럼 어떻게 됩니까', 'Main Topic: 주제: 코파일럿의 변화\nSubtopic: 주어진 텍스트는 짧고 구체적인 내용이 부족하여 섹션으로 나누기 어려운 점이 있습니다. 하지만 "코파일럿"에 대한 일반적인 주제와 관련된 서브토픽을 제안할 수 있습니다. 다음은 코파일럿에 관련된 가능한 서브토픽들입니다:\n\n참고로 코파일럿이 뭔가 바뀌는 것 같아요', 'Main Topic: 주제: 코파일럿\nSubtopic: "코파일럿 있습니다"라는 짧은 문장은 구체적인 내용이 부족하여 명확한 섹션을 구분하기 어렵습니다. 그러나 이 문장이 특정 주제나 문맥을 가지고 있

In [82]:
documents

[Document(metadata={'start': 0.0, 'end': 8.22, 'main_topic': 'The main topic of the text is about using artificial intelligence to create similar images, specifically aimed at office workers.', 'subtopic': 'Sure! Here are the extracted subtopics for each section of the provided text:'}, page_content='전 감자이구요 직장인을 위한 인공지능 오늘은 비슷한 이미지를 만드는 방법 알려드리려고 합니다'),
 Document(metadata={'start': 8.22, 'end': 14.16, 'main_topic': '주제: 인공지능을 활용한 이미지 생성 방법', 'subtopic': 'Sure! Here are some possible subtopics based on the provided text:'}, page_content='앞에 영상에서 우리는 인공지능 서비스들로 쉽게 이미지 만드는 방법을 알려드렸는데요'),
 Document(metadata={'start': 14.16, 'end': 21.92, 'main_topic': '주요 주제: 원하는 이미지 생성 및 가공 방법.', 'subtopic': '물론입니다! 제공된 텍스트를 기반으로 각 섹션의 주제를 다음과 같이 나누어 볼 수 있습니다:'}, page_content='이번에는 내가 원하는 이미지와 비슷한 이미지를 만들고 이를 가공하는 방법을 알려드리겠습니다'),
 Document(metadata={'start': 21.92, 'end': 26.34, 'main_topic': '주요 주제는 "채트 GPT 서비스"입니다.', 'subtopic': 'Sure! Here are potential subtopics based on the provided text:'}, page_co